#### Задача 1 (10 баллов). 

Напишите программу, которая будет автоматически обрабатывать сырой корпус текстов. Что это должно быть:

- у нас есть некоторое количество текстовых файлов, которые лежат рассортированными по папкам. Вложенных подпапок может быть много. 
- мы хотим вытащить тексты из всех этих файлов, разобрать их (можно использовать парсер spacy или stanza) и позаписать в новой директории одноименные conllu. 
- Пользователь при старте программы указывает путь к главной директории исходных файлов и желаемое название папки с аутпутом. Мы должны: а) проверить, что директория с исходниками существует; б) что пользователь не ввел название, в котором есть :, ? и подобные символы, которые запрещены системой в названиях файлов. 
- метод, который будет заниматься открыванием файлов и считыванием текста из них, должен не вываливать ошибку в случае, если у нас файл не в той кодировке, а только делать предупреждение и пропускать файл. 
- при этом должны вестись логи с именами пропущенных файлов. Логи записываем в errors.txt.
- естественно, все должно быть в классах. Примерная структура может быть такой:

In [1]:
import os
# from pathlib import Path
import re
import spacy
from conllu import serializer
from conllu import TokenList
import logging

# также я отдельно установила модель ru_core_news_sm

In [2]:
class CorpusCreator:
    def __init__(self, input_dir, output_dir):

        '''инициализируем пути'''
        self.input_dir = os.path.abspath(input_dir)
        self.output_dir = os.path.abspath(output_dir)
                
        '''проверки на существование директории и допустимые символы'''
        if not os.path.isdir(input_dir):
            raise ValueError('Указанной директории не существует.')
        
        invalid_chars = r'[<>?*"]'
        if re.search(invalid_chars, self.output_dir):
            raise ValueError('В названии папки для выходных данных содержатся недопустимые символы.')
        

        os.makedirs(output_dir, exist_ok=True) # создаем выходную директорию, если её нет

        '''заводим файл с логами'''
        logging.basicConfig(
            level=logging.WARNING, # записываются логи предупреждения и ошибки
            filename=os.path.join(self.output_dir, 'errors.txt'), 
            filemode='w', 
            format='%(asctime)s — %(message)s'
            )
        
        self.nlp = spacy.load('ru_core_news_sm')

    def openfile(self, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except UnicodeEncodeError:
            logging.warning(f'Неверная кодировка файла {file_path}. Файл пропущен.')
            return None
    
    def parse(self, text):
        return self.nlp(text)
        
    def writefile(self, data, output_file):
        with open(output_file, 'w', encoding='utf-8') as file:
            for sent in data.sents:
                conllu_data = []
                for token in sent:
                    feats = str(token.morph) if token.morph else "_"
                    conllu_data.append({
                        'id': token.i + 1,
                        'form': token.text,
                        'lemma': token.lemma_,
                        'upostag': token.pos_,
                        'xpostag': '_',
                        'feats': feats,
                        'head': token.head.i + 1 if token.head.i != token.i else 0,
                        'deprel': token.dep_,
                        'deps': '_',
                        'misc': '_',
                    })
                
                token_list = TokenList(conllu_data, metadata={'text': sent.text})
                file.write(serializer.serialize(token_list) + "\n\n")
    
    def process(self):
        '''os.walk() возвращает кортеж, состоящий из:
        1. root: текущий путь директории, которую обходит os.walk()
        2. _ : список подпапок текущей директории (нам не нужен этот список, поэтому никак его не обозначаем)
        3. files: файлы, находящиеся в текущей директории'''
        for root, _, files in os.walk(self.input_dir):
            for file_name in files:                             # итерируемся по каждому файлу (под)директорий
                input_file_path = os.path.join(root, file_name) # склеиваем название файла с остальным путем и получаем входной путь
        
                relative_path = os.path.relpath(input_file_path, self.input_dir)
                file_name = os.path.splitext(os.path.basename(input_file_path))[0]  # имя файла без расширения
                output_file_path = os.path.join(
                    self.output_dir, os.path.splitext(relative_path)[0] + f'{file_name}.conllu'
                )

                os.makedirs(os.path.dirname(output_file_path), exist_ok=True)   # создаем промежуточные директории для выходного файла

                text = self.openfile(input_file_path)
                if text is None:
                    continue

                parsed_data = self.parse(text)
                self.writefile(parsed_data, output_file_path)
                print(f'Обработан файл: {input_file_path} ====> {output_file_path}')


In [3]:
input_dir = r"C:\Users\PC\универ\mag\prog_mag_hw\2nd year\texts"
output_dir = r"C:\Users\PC\универ\mag\prog_mag_hw\2nd year\corpus"

In [4]:
corpus_creator = CorpusCreator(input_dir, output_dir)

In [5]:
corpus_creator.process()

Обработан файл: C:\Users\PC\универ\mag\prog_mag_hw\2nd year\texts\file1.txt ====> C:\Users\PC\универ\mag\prog_mag_hw\2nd year\corpus\file1file1.conllu
Обработан файл: C:\Users\PC\универ\mag\prog_mag_hw\2nd year\texts\file2.txt ====> C:\Users\PC\универ\mag\prog_mag_hw\2nd year\corpus\file2file2.conllu
Обработан файл: C:\Users\PC\универ\mag\prog_mag_hw\2nd year\texts\folder1\file3.txt ====> C:\Users\PC\универ\mag\prog_mag_hw\2nd year\corpus\folder1\file3file3.conllu
Обработан файл: C:\Users\PC\универ\mag\prog_mag_hw\2nd year\texts\folder1\folder2\subfolder\file4.txt ====> C:\Users\PC\универ\mag\prog_mag_hw\2nd year\corpus\folder1\folder2\subfolder\file4file4.conllu


На дополнительный балл: изучите библиотеку argparse и используйте ее в своей программе таким образом, чтобы ее можно было вызывать с параметрами из консоли. То есть, в консоли пишете что-то типа: 
    
    python mycorpusreader.py C:\mycorpusraw output